In [1]:

import torch.nn as nn

from transformers import AutoConfig, AutoTokenizer



In [2]:
tokenizer_model_name_path="hfl/chinese-roberta-wwm-ext"

tokenizer = AutoTokenizer.from_pretrained(tokenizer_model_name_path)


In [35]:
x = "投陋的小明的成基差劫极了。"

In [36]:
from transformers import BertForMaskedLM

model = BertForMaskedLM.from_pretrained("/remote-home/xtzhang/CTC/CTC2021/SE_tmp_back/tmp/sighan/bert_MaskedLM_base_raw.epoch10.bs128")


In [37]:
tmp = tokenizer(x)
import torch
source = { k:torch.tensor([v]) for k,v in tmp.items() }

In [38]:
print(tmp)

{'input_ids': [101, 2832, 7358, 4638, 2207, 3209, 4638, 2768, 1825, 2345, 1223, 3353, 749, 511, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [39]:
preds = torch.softmax(model(**source).logits, dim=2)

In [40]:
print(preds.shape)

torch.Size([1, 15, 21128])


In [44]:
#c = torch.argmax(q, 2)
p = torch.topk(preds, 10)[0][0][1]
c = torch.topk(preds, 10)[-1][0][1]

print(p)
print(c)

for o, i in enumerate(c):
    print(p[o], tokenizer.decode(i.numpy().tolist()))

tensor([0.4995, 0.2312, 0.1092, 0.0425, 0.0314, 0.0203, 0.0073, 0.0071, 0.0061,
        0.0059], grad_fn=<SelectBackward0>)
tensor([5110, 5042,  682, 2577, 1928, 1936, 1159, 3193, 2405, 6577])
tensor(0.4995, grad_fn=<SelectBackward0>) 粗
tensor(0.2312, grad_fn=<SelectBackward0>) 简
tensor(0.1092, grad_fn=<SelectBackward0>) 丑
tensor(0.0425, grad_fn=<SelectBackward0>) 怀
tensor(0.0314, grad_fn=<SelectBackward0>) 头
tensor(0.0203, grad_fn=<SelectBackward0>) 奇
tensor(0.0073, grad_fn=<SelectBackward0>) 初
tensor(0.0071, grad_fn=<SelectBackward0>) 早
tensor(0.0061, grad_fn=<SelectBackward0>) 幼
tensor(0.0059, grad_fn=<SelectBackward0>) 贫


In [42]:
pred = torch.argmax(preds, dim=2)
print(tokenizer.batch_decode(pred))

['[CLS] 粗 陋 的 小 明 的 成 绩 差 劲 极 了 。 [SEP]']


In [48]:
print(tokenizer.batch_decode(source["input_ids"]))

['[CLS] 投 陋 的 小 明 的 成 基 差 劫 极 了 。 [SEP]']


In [49]:
print(pred)
print(source["input_ids"])

tensor([[ 101, 5110, 7358, 4638, 2207, 3209, 4638, 2768, 5327, 2345, 1226, 3353,
          749,  511,  102]])
tensor([[ 101, 2832, 7358, 4638, 2207, 3209, 4638, 2768, 1825, 2345, 1223, 3353,
          749,  511,  102]])


In [61]:
path = "./data/confusion.txt"

import reader
confusion_reader = reader.ConfusionSetReader()

confusion_reader.run()


[INFO] [Reader] [Building Confusion]
[INFO] [Reader] [Building Graph]
[INFO] [Reader] [Init Vocab]


In [63]:
confusion_reader.graph["投"]

['又', '透', '壹', '扭', '技', '丑', '援', '役', '没', '偷', '窦', '设', '头', '昱', '股']

In [85]:
z = torch.topk(preds, 5)[1][0][1]
print(z)

tensor([5110, 5042,  682, 2577, 1928])


In [75]:
def get_sim(i):
    confusion = confusion_reader.graph[tokenizer.decode(i)]
    return [ tokenizer.convert_tokens_to_ids(j) for j in confusion]

print(get_sim(2832))

[1348, 6851, 1902, 2814, 2825, 682, 3001, 2514, 3766, 982, 4977, 6392, 1928, 3222, 5500]


In [88]:
print(pred[0][0])
print(source["input_ids"][0])

tensor(101)
tensor([ 101, 2832, 7358, 4638, 2207, 3209, 4638, 2768, 1825, 2345, 1223, 3353,
         749,  511,  102])


In [93]:
tokenizer.convert_tokens_to_ids("得")#4638, 1765, 2533

2533

In [90]:
p = torch.topk(preds, 5)[0][0]
c = torch.topk(preds, 5)[-1][0]

new_preds = []

for i in range(len(pred[0])):
    if pred[0][i] != source["input_ids"][0][i]:
        confusion = get_sim(source["input_ids"][0][i])
        for j in range(len(c[i])):
            if c[i][j] in confusion:
                new_preds.append(c[i][j])
                break

    else:
        new_preds.append(pred[0][i])

print(tokenizer.decode(new_preds))

[CLS] 丑 陋 的 小 明 的 成 绩 差 劲 极 了 。 [SEP]
